## Transform Awards Column

Award data comes in a string format. Format is quite consistent so a parser was constructed in order to pull out the integers.

In [1]:
import pandas as pd
import os

In [2]:
path = os.path.join('..', '..', 'resources','cleaned_data', 'movies_complete_cleaned.csv')
df = pd.read_csv(path)
selected = ['name', 'awards']
df = df[selected]
df

,name,awards
0,Doctor Strange,Nominated for 1 Oscar. Another 19 wins & 67 no...
1,Sleight,3 nominations.
2,Silence,Nominated for 1 Oscar. Another 6 wins & 56 nom...
3,Manchester by the Sea,Won 2 Oscars. Another 127 wins & 263 nominations.
4,Dirty Grandpa,2 wins & 11 nominations
...,...,...
6251,Hoosiers,Nominated for 2 Oscars. Another 3 wins & 3 nom...
6252,Off Beat,7 wins & 3 nominations.
6253,Big Trouble in Little China,1 win & 1 nomination.
6254,Biggles: Adventures in Time,NaN


In [3]:
df_new = df.copy()

df_new["oscar wins and nominations"] = ''
df_new["other wins"] = ''
df_new["other nominations"] = ''
df_new["total awards"] = ''

for i, row in df.iterrows():
    a_string = row[1]
    
    oscar = 0
    wins = 0
    noms = 0
  
    try:
        splitted = a_string.split()
        ints = [int(i) for i in a_string.split() if i.isdigit()]
#         print(f"{a_string}||{len(ints)}")
        if len(ints) == 1:
            if splitted[1].strip("s.")== "nomination":
                noms = ints[0]
#                 print(f"NOM !!{a_string}")
            elif splitted[1].strip("s.")== "win":
                wins = ints[0]
#                 print(f"WIN !!{a_string}")

        elif len(ints) == 2:
            if "Another" not in splitted:
                noms = ints[1]
                wins = ints[0] 

            else:
                if "Oscar." in splitted or "Oscars." in splitted:
                    oscar = ints[0] 
                    if splitted[-1].strip("s.") == "nomination":
                        noms = ints[1] 
#                         print(f"{a_string}||{ints[0]} oscar wins+noms,{ints[1]} nomination(s)")
                    elif splitted[-1].strip("s.") == "win":
                        wins = ints[1] #other wins
#                         print(f"{a_string}||{ints[0]} oscar wins+noms, {ints[1]} wins(s)")
                else:
                    if splitted[-1].strip("s.") == "nomination":
                        noms = ints[0]+ints[1] #noms
#                         print(f"{a_string}||{ints[0]+ints[1]} nomination(s)")
                    elif splitted[-1].strip("s.") == "win":
                        noms = ints[0] #other noms
                        wins = ints[1] #wins
#                         print(f"{a_string}||{ints[0]} nomination(s) and {ints[1]} win(s)")

        elif len(ints) == 3:
            if "Oscar." in splitted or "Oscars." in splitted:
                oscar = ints[0] 
                noms = ints[2]
                wins = ints[1]  
#                 print(f"{a_string}|| {oscar} oscar wins/noms. {wins} wins. {noms} noms.")
            else:
#                 print(a_string)
                if splitted[0] == "Nominated":
                    noms = ints[0] + ints[2]
                    wins = ints[1]
                elif splitted[0] == "Won":                    
                    noms = ints[2]
                    wins = ints[0] + ints[1]
#                 print(f"{a_string}|| {wins} wins. {noms} noms.")

    except AttributeError:
        pass
        
    df_new.loc[i,"oscar wins and nominations"] = oscar
    df_new.loc[i,"other wins"] = wins
    df_new.loc[i,"other nominations"] = noms
    df_new.loc[i,"total awards and nominations"] = oscar+wins+noms

#     print(f"{a_string}|| {oscar} oscar wins or nominations. {wins} wins. {noms} nominations.")

    
df_new.head() 


,name,awards,oscar wins and nominations,other wins,other nominations,total awards,total awards and nominations
0,Doctor Strange,Nominated for 1 Oscar. Another 19 wins & 67 no...,1,19,67,,87.0
1,Sleight,3 nominations.,0,0,3,,3.0
2,Silence,Nominated for 1 Oscar. Another 6 wins & 56 nom...,1,6,56,,63.0
3,Manchester by the Sea,Won 2 Oscars. Another 127 wins & 263 nominations.,2,127,263,,392.0
4,Dirty Grandpa,2 wins & 11 nominations,0,2,11,,13.0


In [5]:
path = os.path.join('..', '..', 'resources','cleaned_data', 'parsed_awards_table.csv')
df_new.to_csv(path, index = False)